In [ ]:
import os
import json
import glob
import pandas as pd

In [ ]:
SWEEP_PATH = "../autoencoders/othello_5-21/othello-gated_anneal/"

SAE_TRAINER_TYPE = "gated_anneal"

HPARAMS_DICT = {
    "p_anneal" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
    "p_anneal_new" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
    "standard_new" : ["lr", "l1_penalty", "dict_size"],
    "standard" : ["lr", "l1_penalty"],
    "gated" : ["lr", "l1_penalty", "dict_size"],
    "gated_anneal" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
}
HPARAMS = HPARAMS_DICT[SAE_TRAINER_TYPE]

# read in the `results.csv` file from SWEEP_PATH
df = pd.read_csv(SWEEP_PATH + "results.csv")

# filter for SAE type
df = df[df["autoencoder_path"].str.contains(SAE_TRAINER_TYPE + "/")]

# add trainer_id to data frame
df["trainer_id"] = df["autoencoder_path"].map(lambda p: p.split("/")[-2])
df[["trainer_id", "l0", "frac_recovered"]].sort_values("l0")



In [ ]:
#trainer_dirs = df["autoencoder_path"].to_list()
trainer_dirs = [SWEEP_PATH + t_id + "/" for t_id in df["trainer_id"]]

trainer_configs = {}
for t_dir in trainer_dirs:
    trainer_id = t_dir.split("/")[-2]
    with open(t_dir + "/config.json", "r") as f:
        config = json.load(f)
        trainer_configs[trainer_id] = {}
        for h in HPARAMS:
            trainer_configs[trainer_id][h] = config["trainer"][h]

for h in HPARAMS:
    df[h] = df["trainer_id"].map(lambda t_id : trainer_configs[t_id][h])
    
df[["trainer_id", "l0", "frac_recovered"] + HPARAMS].sort_values("l0")

In [ ]:
config